### The Culture of International Relations

#### About this project
Cultural treaties are the bi-lateral and multilateral agreements among states that promote and regulate cooperation and exchange in the fields of life generally call cultural or intellectual. Although it was only invented in the early twentieth century, this treaty type came to be the fourth most common bilateral treaty in the period 1900-1980 (Poast et al., 2010). In this project, we seek to use several (mostly European) states’ cultural treaties as a historical source with which to explore the emergence of a global concept of culture in the twentieth century. Specifically, the project will investigate the hypothesis that the culture concept, in contrast to earlier ideas of civilization, played a key role in the consolidation of the post-World War II international order.

The central questions that interest me here can be divided into two groups: 
- First, what is the story of the cultural treaty, as a specific tool of international relations, in the twentieth century? What was the historical curve of cultural treaty-making? For example, in which political or ideological constellations do we find (the most) use of cultural treaties? Among which countries, in which historical periods? What networks of relations were thereby created, reinforced, or challenged? 
- Second, what is the "culture" addressed in these treaties? That is, what do the two signatories seem to mean by "culture" in these documents, and what does that tell us about the role that concept played in the international system? How can quantitative work on this dataset advance research questions about the history of concepts?

In this notebook, we deal with these treaties in three ways:
1) quantitative analysis of "metadata" about all bilateral cultural treaties signed betweeen 1919 and 1972, as found in the World Treaty Index or WTI (Poast et al., 2010).
    For more on how exactly we define a "cultural treaty" here, and on other principles of selection, see... [add this, using text now in "WTI quality assurance"].
2) network analysis of the system of international relationships created by these treaties (using data from WTI, as above).
3) Text analysis of the complete texts of selected treaties. 

After some set-up sections, the discussion of the material begins at "Part 1," below.

In [1]:
%%html
<style>
.widget-vbox .widget-label {width: 60px; }
.widget-toggle-button { width: 100px; }
.jupyter-widgets { font-size: 8pt; }
.widget-label { font-size: 8pt;}
.widget-vbox .widget-label { width: 60px; }
.widget-dropdown select { font-size: 8pt; width: 102px; }
.widget-select > select { font-size: 8pt; }
.widget-toggle-buttons .widget-toggle-button { font-size: 8pt; }
.widget-inline-hbox .widget-readout { min-width: 30px; font-size: 8pt; margin-left: 0; text-align: left;}
.widget-hslider.slider-container { width: 80px; }
.widget-slider.widget-hslider { width: 200px; }
</style>

### <span style='color:blue'>**Mandatory Prepare Step**</span>: Setup Notebook


In [2]:
# Setup
%load_ext autoreload
%autoreload 2

import os
import warnings
import ipywidgets as widgets
import pandas as pd
import types

from bokeh.plotting import output_notebook

os.sys.path = os.sys.path if '..' in os.sys.path else os.sys.path + ['..']

warnings.filterwarnings('ignore')

import common.config

from common.color_utility import StaticColorMap
from common.file_utility import FileUtility
from common.widgets_utility import WidgetUtility
from common.utility import extend, flatten, getLogger
from common.treaty_state import load_wti_index, default_period_specification

from network_analysis_gui import display_network_analyis_gui
from network_analysis_plot import plot_network, get_palette
from network_analysis import create_party_network

logger = getLogger(name='cultural_treaties')

wti_index = load_wti_index('../data')

unknown_parties = wti_index.check_party()
logger.warning('[{} UNKNOWN PARTIES] '.format(len(unknown_parties)) + (' '.join([ '"' + str(x) + '"' for x in unknown_parties])))
# display(pd.DataFrame({'party': unknown_parties}))

output_notebook()


2018-10-06 18:10:44,767 : INFO : Data loaded!
2018-10-06 18:10:47,136 : WARNING : [40 UNKNOWN PARTIES] "" "FR. INDOCHINA" "GIBRALTAR" "IDEA" "MANCHUKUO" "PORT. INDIA" "VIETNAM" "SAAR" "HEDJAZ" "BALKAN ENT." "SAAR COM." "NEFC" "SERBIA" "SHAPE" "RHINELAND C" "NEJD" "NIUE" "ADEN" "GERMANY" "BELGIUM- BELG. CONGO" "DANZIG" "CZECH REPUBLIC" "SPAN. GUINEA" "STRAIT" "SABAYA" "PALESTINE" "ELSAL" "FR. EQ. AFRICA" "HOLY SEE" "WINWARD IS." "IGNORE" "BRIT. HONDURAS" "MARTINIQUE" "CURACAO" "USAS" "ATPC" "BELGIUM- RWANDA" "A-E SUDAN" "BRIT.INDIA" "BRIT. GUIANA"


Loading BokehJS ...

### <span style='color: red'>WORK IN PROGRESS</span> Task: Network Visualization of Signed Treaties
        
- **TODO: Button "Recode 7CULT" button**<br>
- **TODO: Add Category and year as edge labels **<br>
- **TODO: Split result into one graph per category**

In [3]:
# Visualize treaties
plot_data = types.SimpleNamespace()

from common.network.layout import layout_network
from common.network.networkx_utility import network_edges_to_dicts, pandas_to_network_edges, get_node_attributes

G, layout = None, None
edges, nodes = None, None
data = None

def display_party_network(
    parties,
    period_group,
    treaty_filter,
    topic_group=None,
    layout_algorithm='',
    C=1.0,
    K=0.10,
    p1=0.10,
    output='network',
    party_name='party',
    node_size_range=[40,60],
    refresh=False,
    palette_name=None,
    width=900,
    height=900,
    node_size=None,
    node_partition=None,
    wti_index=None,
    gui=None
    ):
    global plot_data, G, layout, edges, nodes, data
    try:
        weight_threshold=0.0
        recode_is_cultural = False

        palette = get_palette(palette_name)
        
        gui.refresh.value = False
        gui.progress.value = 1
        
        kwargs = dict(
            period_group=period_group,
            treaty_filter=treaty_filter,
            recode_is_cultural=recode_is_cultural
        )
        
        data = wti_index.get_party_network(party_name, topic_group, parties, **kwargs)
        
        if topic_group is not None:

            # FIXME!!
            data = data.loc[(data.topic.isin(topic_group.keys()))]
            
            #line_palette = pals.Set1[8]
            #group_keys = category_group_settings[category_map_name].keys()
            #line_palette_map = { k: i % len(line_palette) for i, k in enumerate(group_keys) }
            # print(line_palette_map)
            data['line_color'] = 'red' #data.category.apply(lambda x: line_palette[line_palette_map[x]])

        else:
            data['category'] = data.topic
            
        gui.progress.value = 2
        
        #if not multigraph:
        #    data = data.groupby(['party', 'party_other']).size().reset_index().rename(columns={0: 'weight'})
        
        G = create_party_network(data, K, node_partition, palette) #, multigraph)
        
        gui.progress.value = 3

        if output == 'network':

            if weight_threshold > 0:
                G = filter_by_weight(G, weight_threshold)
            
            layout, _ = layout_network(G, layout_algorithm, **dict(scale=1.0, K=K, C=C, p=p1))
            gui.progress.value = 4

            edges = network_edges_to_dicts(G, layout)
            nodes = get_node_attributes(G, layout)
            
            edges = { k: list(edges[k]) for k in edges }
            nodes = { k: list(nodes[k]) for k in nodes }
    
            node_size = node_size if not node_size is None else node_size_range[0]
            node_size = setup_node_size(nodes, node_size, node_size_range)
            
            y_offset = setup_label_y_offset(nodes, node_size)
            text_opts = extend(label_text_opts, dict(y_offset=y_offset, x_offset=0))
            gui.progress.value = 5 
#            data = plot_party_network(
#                nodes=nodes,
#                edges=edges,
#                figsize=(width, height),
#                node_size=node_size,
#                text_opts=text_opts,
#                **network_plot_opts
#            )
            
#            plot_data.__init__(data.__dict__)
            
            gui.progress.value = 6
            gui.time_travel_range.max = len(edges['source'])
            gui.time_travel_range.value = [0, len(edges['source'])]
            
            #bp.show(p)

        elif output == 'table':
            display(data)
        else:
            print('unknown output device: ' + output)
            
    except Exception as ex:
        print(ex)
        raise
    finally:
        pass
        gui.progress.value = 0
        
def update_partial_party_network(range):
    
    global plot_data
    edge_count = len(plot_data.edges['source'])
    edges = { k: plot_data.edges[k][range[0]:range[1]] for k in plot_data.edges } 
    nodes_indices = set(edges['source'] + edges['target'])
    
    df = pd.DataFrame({k:plot_data.nodes[k] for k in plot_data.nodes if isinstance(plot_data.nodes[k], list) }).set_index('id')
    
    nodes = df.loc[nodes_indices].reset_index().to_dict(orient='list')
    nodes = extend(nodes, {k:plot_data.nodes[k] for k in plot_data.nodes if not isinstance(plot_data.nodes[k], list) })

    return nodes, edges
    

display_network_analyis_gui(wti_index, display_party_network, update_partial_party_network)
